## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-10-20-54-35 +0000,nypost,Laura Loomer rips Newsmax CEO Chris Ruddy in s...,https://nypost.com/2025/12/10/business/laura-l...
1,2025-12-10-20-54-26 +0000,bbc,Fed lowers interest rates but future cuts unce...,https://www.bbc.com/news/articles/cx257k3n2g1o...
2,2025-12-10-20-53-00 +0000,wsj,"Treasury Yields, Dollar Fall After Fed Cut, Po...",https://www.wsj.com/finance/investing/jgbs-mix...
3,2025-12-10-20-52-41 +0000,nypost,How to tip your Amazon driver $5 at no cost us...,https://nypost.com/2025/12/10/business/how-to-...
4,2025-12-10-20-51-30 +0000,nyt,A Cinnabon Worker Was Fired for a Racist Slur....,https://www.nytimes.com/2025/12/10/us/cinnabon...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2366/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
53,trump,56
13,fed,21
218,new,19
176,out,17
39,was,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
212,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...,141
261,2025-12-10-09-00-00 +0000,cbc,Is Trump’s stark new security strategy the end...,https://www.cbc.ca/news/world/trump-world-orde...,114
312,2025-12-10-01-42-00 +0000,wsj,Eileen Higgins became the first Democrat to wi...,https://www.wsj.com/politics/elections/democra...,110
274,2025-12-10-07-00-26 +0000,nyt,Trump Revives Attacks on Fed Members Before In...,https://www.nytimes.com/2025/12/09/us/politics...,110
167,2025-12-10-15-51-13 +0000,nypost,Trump’s final round of interviews with Fed cha...,https://nypost.com/2025/12/10/business/trumps-...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
212,141,2025-12-10-11-56-00 +0000,wsj,The next Fed chair will have to walk the tight...,https://www.wsj.com/politics/policy/can-the-ne...
259,66,2025-12-10-09-34-00 +0000,wsj,Social-Media Ban Imposes Brave New World on Au...,https://www.wsj.com/world/social-media-ban-imp...
177,61,2025-12-10-15-11-40 +0000,nypost,Husband charged with murdering Miss Switzerlan...,https://nypost.com/2025/12/10/world-news/husba...
36,54,2025-12-10-20-09-00 +0000,wsj,The Journal answered subscribers’ questions ab...,https://www.wsj.com/economy/central-banking/li...
10,52,2025-12-10-20-46-06 +0000,nyt,Venezuela’s Nobel Peace Prize Winner Raises St...,https://www.nytimes.com/2025/12/10/world/europ...
312,46,2025-12-10-01-42-00 +0000,wsj,Eileen Higgins became the first Democrat to wi...,https://www.wsj.com/politics/elections/democra...
92,46,2025-12-10-18-30-39 +0000,latimes,Congressional Democrats say Paramount's bid fo...,https://www.latimes.com/entertainment-arts/bus...
227,46,2025-12-10-11-03-45 +0000,nypost,DOJ civil rights chief Harmeet Dhillon reveals...,https://nypost.com/2025/12/10/us-news/harmeet-...
317,41,2025-12-10-01-14-00 +0000,wsj,Kevin Hassett says he would rely on his own ju...,https://www.wsj.com/economy/central-banking/ke...
106,38,2025-12-10-17-59-00 +0000,wsj,Citizens of countries including the U.K. and F...,https://www.wsj.com/politics/policy/trump-admi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
